<a href="https://colab.research.google.com/github/bamtak/AI-Playground/blob/master/kolawole_tajudeen_lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chit Chat Chatbots

In the previous lab, we explored models that try to answer questions by reasoning over free-text input. In this lab, we will explore two types of models to create chatbots.

First, let's consider important qualities for a chit-chat chatbot system


1.   **Readability** - whatever model we use, the chats it creates should be easily understood by humans
2.   **Consistency** - when chatting with a chatbot, the bot should maintain consistent information. Imagine a bot that says "Hi I'm Jack'' and then "Hello, my name is Jane" - quite confusing
3.    **Engaging** - To encourage users to talk to the bot, the bot should be able to generate interesting, engaging responses. If the only response was "wow, that's cool," users are quite unlikely to want to talk very much to the chat bot



In [1]:
%%html
<p style='color: blue;'>
  Throughout the lab, there will be <b>questions</b> you should answer. <b>All questions you need to write an answer to will be in this blue color.</b>
  
  <br>Please write brief answers- no need for long explanations. 
  <br>There can be multiple correct answers to the questions.
  
  <br><br>The goal of these questions is to:
  <ul style='color: green;'>
    <li> Review the lecture material in the context of practical models and develop intuition about the models
    <li> Develop a sense of experimentation - we will pretend we have a dataset and will walk through an experimental thought process.
  </ul>

<b>We are going to do the lab as a group. <br>I will explain the sections in more depth, as we did not cover dialogue deeply during the lecture. <br> After we discuss, I will provide time for you to write a few sentences. At the end of the lab, you will hand it in. In theory, everyone should be finished together!</b>

  
</p>

## Data

The dataset we will use for this lab is called `PersonaChat` - it was created to directly address problem 2. Each person talking in the dataset has a personality, which helps maintain consistency in the dialogue. We saw it last week in the tutorials as well (when you worked through beam search)

In [2]:
!git clone https://github.com/facebookresearch/ParlAI.git ~/ParlAI  > /dev/null
!cd ~/ParlAI; python setup.py develop > /dev/null

Cloning into '/root/ParlAI'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 30055 (delta 47), reused 43 (delta 19), pack-reused 29963
Receiving objects: 100% (30055/30055), 57.94 MiB | 24.77 MiB/s, done.
Resolving deltas: 100% (21390/21390), done.
zip_safe flag not set; analyzing archive contents...
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '2019.08.19' to '2019.8.19'
  warnings.warn(tmpl.format(**locals()))
regex_3/_regex.c: In function ‘folded_char_at’:
regex_3/_regex.c:10625:9: warning: variable ‘folded_len’ set but not used [-Wunused-but-set-variable]
     int folded_len;
         ^~~~~~~~~~
regex_3/_regex.c: In function ‘fuzzy_match_group_fld’:
regex_3/_regex.c:11503:10: warning: ‘data.new_text_pos’ may be used uninitialized in this function [-Wmaybe-uninitialized]
     if (!record_fuzzy(state, data.fuzzy_type, data.new_text_pos - dat

**Example: **

your persona: i just started college.

your persona: i have 3 science classes.

your persona: i work part time in the campus library.

your persona: i am living at home but hope to live in the dorms next year.

**Partner Dialogue**: hi how are you doing

**Your Response**: great ! just got off work and relaxing before i study

In [3]:
# let's download and take a look at some examples of data in PersonaChat
!python ~/ParlAI/examples/display_data.py --task personachat --datatype train

[creating task(s): personachat]
[building data: /root/ParlAI/data/Persona-Chat]
[ downloading: http://parl.ai/downloads/personachat/personachat.tgz to /root/ParlAI/data/Persona-Chat/personachat.tgz ]
[ Checksum Successful ]
unpacking personachat.tgz
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
- - - NEW EPISODE: personachat - - -
your persona: i like to remodel homes.
your persona: i like to go hunting.
your persona: i like to shoot a bow.
your persona: my favorite holiday is halloween.
hi , how are you doing ? i am getting ready to do some cheetah chasing to stay in shape .
   you must be very fast . hunting is one of my favorite hobbies .
i am ! for my hobby i like to do canning or some whittling .
   i also remodel homes when i am not out bow hunting .
that is neat . when i was in high school i placed 6th in 100m dash !
   that is awesome . do you have a favorite season or time of year ?
i do not . but i do have a favorite meat since tha

In [4]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What do the personalities look like?</li>
    <li>How does creating bots with these simple personalities address consistency for chatbots? </li>
    <li>What are some drawbacks/limitations of these specific personalities for addressing the problem of consistency?</li>
  </ul>
</p>

**ANSWER**
- A friendly university/college science student
- It addresses consistency by grounding the surface level of conversation
- One of the limitations are:
> - Inadequacy of dialogue knowledge which makes their understanding of human context difficult
 - The nature of personalities in dataset limit what can be achieved by the model and kind of problems to be solved using the chat bots



---


***Let's understand how much data we have. Let's compute the following using ParlAI:***


1.   **How many turns of data do we have?** In dialogue datasets, "amount of data" is measured in dialogue turns. Each time there is a single line of dialogue, that is called a "turn"
2.   **On average, how many words form a model input?**


---




In [5]:
!python ~/ParlAI/parlai/scripts/data_stats.py -t personachat -dt train -ltim 10000

[ note: changing datatype from train to train:ordered ]
[creating task(s): personachat]
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
[ loaded 8939 episodes with a total of 65719 examples ]
23s elapsed:
      %done  \
    100.00%   
      exs  \
    65719   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                 stats  \
    
input:
   utterances: 65719
   avg utterance length: 18.356974390967604
   tokens: 1206402
   unique tokens: 14209
   unique utterances: 64580
labels:
   utterances: 65719
   avg utterance length: 11.929411585690591
   tokens: 783989

## Evaluation

How are dialogue models evaluated?



1.   **Automatic Evaluation**: Hits @ 1, Hits @ 5, Hits @ 10, F1
2.   **Human Evaluation**: Pairing Selection, Human Rating



In [6]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Take some notes about what these metrics are and what they mean here</li>
  </ul>
</p>

**Automatic Evaluation:** It is an automated evaluation of model performance using scripts. It is timeless and does not require human inference during evaluation though it does not capture range of random responses; making it diffult for interpretation. Examples are Hits @ 1, Hits @ 5, Hits @ 10, F1
>- Hits @ 1,5 and 10 are very useful in retrieval-based model. They predict the next word in a bank of human written text with a choice depending on top-$k$ ranking scores (1%, 5% and 10%) i.e random model would be $k$% of the time have correct candidates. It is not suitable for generative model since it is unrestricted.
>-F1 is a measure of a test's accuracy given as 
$$F_1 = \left(1+\beta^2\right)\dfrac{\text{precision * reacall}}{\beta^2\text{precision + recall}}$$

**Human Evaluation** It basically require human judgment (by reading) in determing the model performance and super important as dialogue is open-ending conversation. Examples are Pairing Selection and Human Rating
>- Paring Selection: It involves the use of human choice in selecting a more natural turns of utterances (dialogue) from models' histories of dialogue.
- Human Rating: It is the human scoring or scaling (1-5/10) of model's dialogue measuring its readability, relevance, clarity, gramaticality, quality, meaning preserverance, among others.

## Models

There are two main kinds of dialogue models. 

*Retrieval* Models analyze the current dialogue context and try to find appropriate responses in the dataset.

*Generative* Models analyze the current dialogue context
and try to write an answer, word by word, from left to right.
This can be thought of as an application of sequence-to-sequence models,  where the "encoder side" is the dialogue history and the "decoder side" is the dialogue response your chatbot should generate.

In [7]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Let's discuss the pros/cons of retrieval compared to generative models - Are there settings when you might want to use one over the other?</li>
    <li>Compare a chit-chat application to something like booking a movie ticket- would you want to use generative, retrieval, or something else to accomplish that task? Why?</li>
    <li>How can you evaluate generative models with the metrics we discussed before? How do you think they will perform compared to retrieval models?</li>
    <li>In lecture, Antoine mentioned issues with generative model generation being generic and short. How does this happen in beam search?</li>
</ul>

<b> If you would like me to discuss how to actually use generative models in dialog, please say something! Otherwise, we will skip.</b>
</p>

**ANSWER**
______
- Retrieval Model
>- Pros
>> - Literal human utterances; various expresssions with great diversity
>>- Don't make grammatical mistakes because of its handcrafted responses repository
>- Cons
>> - It is not tailored to queries; bottleneck is the size of repository
>>- It is unable to handle unseen cases for which no appropriate predefined response exits.
>>- It can't refer back to contextual entity information in dialogue
>>- It highly depend on data for better performance
- Generative Model
>- Pros
>> - It is tailored for queries; highly coherent
>> - It can refere backs to contextual entity information in dialogue
>>- It generalizes well when testing for unseen cases during training
>- Cons
>> - Insufficient information; universal sentences
>>- It is highly prone to making grammatical mistakes (especially on longer sentences) and besides makes a super generic and short prediction when the model has no real repoonses to queries

- Generative model can be evaluated using the F1 metric evaluation. The comparison is largely depend on the specific model quality and information retrieval where domain of knowledge is different in training and testing

- Depending on the size of Beam Search as beam search predicts probable next candidate; maximizing the probability over the sentence where "rare or interesting" sentence is penalized by the beam search for such unseen or less seen corpus in training
______

### Retrieval Models

Let's train a model to do retrieval first. We will try the *Memory Net.* 

In [0]:
# We can train a model with the following command: 
# !python ~/ParlAI/examples/train_model.py -m kv_memnn -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy

# but we have limited time in the tutorial, so let's use an already pretrained model

Quick Parameter Refresher:


*  `-m ` means which model we're going to use. Recall retrieval models are trained to rank the true response higher over a set of potential responses from the dataset (in ParlAI, these are called the "label candidates"). When it's time to write a dialogue response, the retrieval model returns the response that is ranked the highest
*  -`t` refers to the task. Here, we are training on PersonaChat data.
* `-dt` refers to the data split. We want to train our model, so we are using the training set.
* `-veps` refers to how often we should evaluate during training, our performance on validation. recall this is important because models, particularly neural ones, have the capacity to memorize the training dataset. So it's important to check how the model is doing on the validation set.
* `--model-file` refers to when your model is saved, what should the filename be
*  `-vmt` refers to the metric which we'll use to decide which model is the best. We'll cover this in the next section





**Let's interact with the model to get a sense of what it's learning. **How is this chat going to work?



1.   You will be assigned a persona. You will chat to the model by typing in the chat box.
2.   The chatbot also has a persona. It's secret and hidden from you!
3.   When you've finished chatting with this bot, type [DONE] and a new model persona will be assigned to the bot, so you can talk to a new bot. 
4.   When you move on to the next chatbot persona, the previous persona will be revealed. 

Interact with the chatbots and the personas. **Try to think about the following:**

*   Do the chatbots follow their persona a lot?
*   Was it difficult to follow your persona?





In [9]:
!python ~/ParlAI/projects/convai2/interactive.py -mf models:convai2/kvmemnn/model

[building data: /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz]
[ downloading: http://parl.ai/downloads/_models/convai2/kvmemnn.tgz to /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz ]
unpacking kvmemnn.tgz
[ warning: overriding opt['model_file'] to /root/ParlAI/data/models/convai2/kvmemnn/model (previously: /checkpoint/jase/20180328/kvmemnn_sweep10/persona-self_rephraseTrn-True_rephraseTst-False_lr-0.1_esz-2000_margin-0.1_tfidf-False_shareEmb-True_hops1_lins0/model )]
[ creating KvmemnnAgent ]
Dictionary: loading dictionary from /root/ParlAI/data/models/convai2/kvmemnn/model.dict
[ num words =  19153 ]
Loading existing model params from /root/ParlAI/data/models/convai2/kvmemnn/model
[loading candidates: /root/ParlAI/data/models/convai2/kvmemnn/model.candspair*]
[caching..]
=init done=
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.form

In [10]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What does this model seem to be doing well? What is it doing poorly? </li>
    <li>Why might it be performing poorly? What kind of experiment could you design to test your hypothesis?</li>
    <li>How do we know if we need to use a more complex model? Would we always want to use a more complex model? Why or why not?</li>
  </ul>
</p>

What does this model seem to be doing well?
- The response seems to be gramatically correct. The responses are meaningful

What is it doing poorly?
- There is no correlation between the response and the previous chat history. It seems like a random response

Why might it be performing poorly?
- It is not using the chat history to predict response. The model we are using didn't seem to learn anything. Using a complex model may help.

What kind of experiment could you design to test your hypothesis?
- We can run the model with random chat and observe the response. If there is no correlation between the responses and the chat messages, then it shows that response are just been generated randomly.

How do we know if we need to use a more complex model?
- We know we need to use a complex model when the model is not performing well on the task or when it performance is below the baseline model.

Would we always want to use a more complex model? Why or why not?
- We do not always need to use a complex model it all depends on the task at hand. It's better to start with a simple model and then increase the complexity in order to give a better result on the task in consideration.
Simple model will be easy to train but may perform poorly on some task while complex model is hard train and not always guanratee to perform well on many tasks.


In [0]:
# Here is a command to train a Transformer Ranker model if you would like to try it out
# !python ~/ParlAI/examples/train_model.py -m transformer/ranker -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy



---


An important aspect of training models is analyzing them. ***Try to answer the following questions.*** 

---




In [12]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Are the models using the persona that we have provided? How can you tell? If I asked you to prove it to me, what experiments could you conduct? </li>
    <li>Previously, we computed some statistics about how long the persona is in the training data. The model has also only seen words present in the training dataset. But what happens if you push the model outside of what data it's been trained on? What kind of performance do you get? Why does this happen, and what could you do if you wanted to improve the model's ability to generalize? </li>
    <li>In ParlAI, we've set the parameters to save the model's best performance based on validation accuracy. What would happen if we saved the model based on the best training accuracy? Why does this happen? (if you like, try this out on your own and see the effect when you interact with the bot)</li>
  </ul>

<b> If you would like me to discuss how to use BERT in dialog, please say something! Otherwise, we will skip.</b>
</p>

Are the models using the persona that we have provided? How can you tell? 
- The model is not using the persona that is provided
- This can be tell because the response is not related to the personal data provided

Previously, we computed some statistics about how long the persona is in the training data. The model has also only seen words present in the training dataset. But what happens if you push the model outside of what data it's been trained on? What kind of performance do you get? Why does this happen, and what could you do if you wanted to improve the model's ability to generalize?
- If we push the model outside of the data it's been trained on, it will not be able to generalise on that. The output may greatly deviate from the expected response.
- In order to improve this, we can make use of model that has been pretrained on open question dialog system.



### [for self exploration] Generative Models

Generative models must produce word for word what they are going to say next in the dialogue. When predicting the next word, it produces a probability distribution over the entire vocabulary space for which word to generate next. To reduce the vocabulary space, we will use **byte-pair encoding** (BPE). 

*How does BPE work?* The BPE algorithm takes as input the training data and the number of *operations* it can do. It passes over the training set and tries to create sub-word units. For example, the word "beautiful" might be split into "beau" "ti" "ful". Each time it splits a word into sub-words, that is one operation. The final vocabulary output consists of these subwords. So "ful" can be part of "beautiful" and part of "fruitful" and so on.


**Questions to ask yourself**:


1.   Why is it important to keep the vocabulary space small?
2.   What does perplexity measure? Why would we use it as a training objective? 




In [0]:
# !python ~/ParlAI/examples/train_model.py -m transformer/generator -t personachat -dt train -veps 0.25 --model-file persona_chat_generative_model -vmt ppl

## Final Thoughts

**What did we learn about dialogue modeling? Review Questions to ask yourself**

*   How do retrieval models work? What about generative? What are their pros and cons?
*   What are some important traits of dialogue systems? How might the traits differ for different dialogue tasks?


**General Takeaways about Machine Learning and Experimentation:**

*   We don't try models just to try them - try to have a reason for conducting an experiment. As we did in the lab, try to analyze what's working well in your models and working poorly. Try to use these reasons to guide why you might want to try other models. Complex is not necessarily better. 
*   Certain models can be better for certain tasks. As we've seen, generative models are working really well for tasks such as machine translation, but have a bit to go before becoming general purpose dialogue generators. 



**I'm really interested in dialogue! What can I do to learn more?**


*   Play around in ParlAI: ParlAI is a general library with many great dialogue models and code for them. It also provides a standard interface to access datasets and interact with various models. 
*   Read the PersonaChat Paper: https://arxiv.org/pdf/1801.07243.pdf
*   Dialog using knowledge: One challenge of these chit chat systems is they do not concretely know any facts. So if you want to chat about a specific topic, the models cannot produce any relevant information - they say generic utterances or incorrect facts. One way to remedy this is to incorporate **knowledge** into the dialogue agents. This has been investigated in many different ways, but one of the first papers to show this is https://arxiv.org/abs/1811.01241. In this work, data is collected by asking one speaker to reference Wikipedia sentences.
* Dialog with BERT: pretty new,  there is an investigation of two ways to use BERT in this paper: https://arxiv.org/abs/1903.03094. 


